In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split

In [ ]:
FILE_PATH = 'assets/' 
MODEL_PATH = 'models/'

In [ ]:
# 전처리에 nltk의 stopwords 활용
try:
    from nltk.corpus import stopwords
    english_stopwords = stopwords.words('english')
except ImportError:
    print("경고: nltk 라이브러리가 설치되지 않았거나 'stopwords'가 다운로드되지 않았습니다.")
    english_stopwords = set()

In [ ]:
# 서브레딧 및 그룹 정의
SUB_PER_GROUP = 4
SUBREDDITS = [
    'Thetruthishere', 'Glitch_in_the_Matrix', 'UnresolvedMysteries', 'Paranormal',
    'learnprogramming', 'cscareerquestions', 'SideProject', 'AskProgramming',
    'TrueFilm', 'booksuggestions', 'TrueGaming', 'LetsTalkMusic',
    'relationship_advice', 'AmItheAsshole', 'offmychest', 'Advice',
    'personalfinance', 'investing', 'Frugal', 'financialindependence',
]
GROUP_MAP = {
    'Mystery': SUBREDDITS[0:SUB_PER_GROUP], 
    'Dev': SUBREDDITS[SUB_PER_GROUP:2*SUB_PER_GROUP], 
    'Culture': SUBREDDITS[2*SUB_PER_GROUP:3*SUB_PER_GROUP], 
    'Life': SUBREDDITS[3*SUB_PER_GROUP:4*SUB_PER_GROUP],
    'Finance': SUBREDDITS[4*SUB_PER_GROUP:5*SUB_PER_GROUP],
}
VECTOR_DIMENSION = 5000  # 문서 벡터 차원

In [ ]:
# 데이터 로드
df = pd.read_csv(FILE_PATH + 'reddit_posts.csv')

subreddit_df = df[df['subreddit'] == SUBREDDITS[0]].copy()
subreddit_df.head(5)

In [ ]:
for subreddit in SUBREDDITS:
    subreddit_df = df[df['subreddit'] == subreddit]
    total_count = len(subreddit_df)
    nan_count = subreddit_df['text'].isna().sum()
    print(f"--- r/{subreddit} ---")
    print(f"전체 데이터 개수: {total_count}")
    print(f"text가 NaN인 데이터 개수: {nan_count}\n")

In [ ]:
# 결측값 제거
print(f"총 결측값 개수: {df['text'].isna().sum()}")
df.dropna(subset=['text'], inplace=True)
print(f"결측값 제거 후 결측값 개수: {df['text'].isna().sum()}")  # 결측값이 제거되었는지 확인

In [ ]:
# 효율적인 불용어 처리를 위해 리스트를 집합(set)으로 변환
stopwords_set = set(english_stopwords)

def preprocess_text(text: str) -> str:
    """
    텍스트 데이터를 전처리하는 함수:
    1. 소문자 변환
    2. 알파벳과 공백을 제외한 모든 문자 제거
    3. 불용어 제거
    """
    # 입력값이 문자열이 아닌 경우 빈 문자열 반환
    if not isinstance(text, str):
        return np.nan
    
    # url 제거
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # 정규표현식을 사용하여 알파벳과 공백 외의 문자 제거 및 소문자 변환
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    
    # 공백을 기준으로 단어 토큰화 후 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords_set]

    # 결과가 없으면(모두 불용어거나 특수문자였으면) NaN 반환
    if not filtered_words:
        return np.nan
    
    # 처리된 단어들을 다시 하나의 문자열로 결합
    return " ".join(filtered_words)

preprocessed_df = pd.read_csv(FILE_PATH + 'reddit_posts.csv')

# 'title'과 'text' 열을 결합하여 'content' 열 생성
preprocessed_df['content'] = preprocessed_df['title'] + " " + preprocessed_df['text']

# 'content' 열에 전처리 함수 적용하여 'processed_content' 열 생성
preprocessed_df['preprocessed_content'] = preprocessed_df['content'].apply(preprocess_text)

# 원본 'content', 'title', 'text' 열 제거
preprocessed_df.drop(columns=['content', 'title', 'text'], inplace=True)

# 전처리 후 결측값 제거
preprocessed_df.dropna(subset=['preprocessed_content'], inplace=True)

# 데이터 분할 (8:2 비율)
train_df, test_df = train_test_split(
    preprocessed_df, 
    test_size=0.2, 
    random_state=42, 
    stratify=preprocessed_df['subreddit']
)

# 분할된 데이터 저장
train_df.to_csv(FILE_PATH + 'train_data.csv', index=False)
test_df.to_csv(FILE_PATH + 'test_data.csv', index=False)

# 전처리 결과 확인
print(f"\n[저장 완료]")
print(f"Train 데이터 개수: {len(train_df)} -> {FILE_PATH}train_data.csv")
print(f"Test 데이터 개수: {len(test_df)} -> {FILE_PATH}test_data.csv")

# Train 데이터 상위 5개 확인
print("\n[Train Data Head]")
print(train_df.head(5))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# 벡터라이저 저장
vectorizer = TfidfVectorizer(max_features=VECTOR_DIMENSION, ngram_range=(1, 2))
vectorizer.fit_transform(train_df['preprocessed_content']).toarray()

joblib.dump(vectorizer, MODEL_PATH + 'tfidf_vectorizer.pkl')